In [1]:
from transformers import BertModel, DistilBertModel
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import datasets
import numpy as np
import pickle as pkl
import os
from os.path import join as oj
from spacy.lang.en import English
import argparse
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegressionCV
from collections import defaultdict
from copy import deepcopy
from tqdm import tqdm
import pandas as pd
from datasets import load_from_disk
import sklearn
import warnings
import config

2022-08-12 19:52:59.432688: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-12 19:52:59.432750: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/accounts/projects/vision/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
2022-08-12 19:53:09.893612: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08

We're going to aggregate embeddings run for a particular order of n-gram here (we'll just add them up).

In [2]:
for dset in sorted(os.listdir(config.data_dir))[::-1]:
    processed_dir = oj(config.data_dir, dset)
    dir_names = [f for f in sorted(os.listdir(processed_dir))
                 if not '-all' in f
                ]

    for i in tqdm(range(len(dir_names))):
        s = dir_names[i]
        start = s.index('=') + 1
        end = s.index('_')
        num = int(s[start: end])
#         if num > 1:
        print('Trying', s)
        s_new = s + '-all'
        if os.path.exists(oj(processed_dir, s_new)):
            print('\tdone already!')
            continue

        pre = s[:start]
        end = s[end:]
        all_exist = True
        num_missing = []
        for num_small in range(1, num):
            fname_small = pre + str(num_small) + end
            all_exist = all_exist and (fname_small in dir_names)
            if not fname_small in dir_names:
                num_missing.append(num_small)

        if not all_exist:
            print('\tmissing small_ngrams: ', num_missing)
            continue

        # load dset
        reloaded_dataset = load_from_disk(oj(processed_dir, s))
        X_train = np.array(reloaded_dataset['train']['embs']).squeeze()
        X_val = np.array(reloaded_dataset['validation']['embs']).squeeze()

        for num_small in range(1, num + 1):
            fname_small = pre + str(num_small) + end            
            reloaded_dataset = load_from_disk(oj(processed_dir, fname_small))
            X_train_small = np.array(reloaded_dataset['train']['embs']).squeeze()
            X_val_small = np.array(reloaded_dataset['validation']['embs']).squeeze()

            X_train += X_train_small
            X_val += X_val_small

        os.makedirs(oj(processed_dir, s_new), exist_ok=True)
        mu = X_train.mean(axis=0)
        sigma = X_train.std(axis=0)
        r = {
            'X_train': X_train,
            'X_val': X_val,
            'mean': mu,
            'sigma': sigma,
        }
        pkl.dump(r, open(oj(processed_dir, s_new, 'data.pkl'), 'wb'))
        print('\tsuccess!')

100%|██████████| 42/42 [00:00<00:00, 16288.56it/s]


Trying ngram=1_sub=-1_bert-base-uncased
	done already!
Trying ngram=1_sub=-1_bert-base-uncased__last_hidden_state_mean
	done already!
Trying ngram=1_sub=-1_distilbert-base-uncased__last_hidden_state_mean
	done already!
Trying ngram=1_sub=-1_philschmid-BERT-tweet-eval-emotion
	done already!
Trying ngram=1_sub=-1_philschmid-BERT-tweet-eval-emotion__last_hidden_state_mean
	done already!
Trying ngram=1_sub=-1_philschmid-DistilBERT-tweet-eval-emotion__last_hidden_state_mean
	done already!
Trying ngram=2_sub=-1_bert-base-uncased
	done already!
Trying ngram=2_sub=-1_bert-base-uncased__last_hidden_state_mean
	done already!
Trying ngram=2_sub=-1_distilbert-base-uncased__last_hidden_state_mean
	done already!
Trying ngram=2_sub=-1_philschmid-BERT-tweet-eval-emotion
	done already!
Trying ngram=2_sub=-1_philschmid-BERT-tweet-eval-emotion__last_hidden_state_mean
	done already!
Trying ngram=2_sub=-1_philschmid-DistilBERT-tweet-eval-emotion__last_hidden_state_mean
	done already!
Trying ngram=3_sub=-1_

100%|██████████| 82/82 [00:00<00:00, 16495.58it/s]


Trying ngram=10_sub=-1_textattack-bert-base-uncased-SST-2
	done already!
Trying ngram=10_sub=1000_bert-base-uncased
	missing small_ngrams:  [9]
Trying ngram=10_sub=100_bert-base-uncased
	missing small_ngrams:  [9]
Trying ngram=10_sub=100_textattack-bert-base-uncased-SST-2
	missing small_ngrams:  [9]
Trying ngram=1_sub=-1_bert-base-uncased
	done already!
Trying ngram=1_sub=-1_bert-base-uncased__last_hidden_state_mean
	done already!
Trying ngram=1_sub=-1_distilbert-base-uncased-finetuned-sst-2-english__last_hidden_state_mean
	done already!
Trying ngram=1_sub=-1_distilbert-base-uncased__last_hidden_state_mean
	done already!
Trying ngram=1_sub=-1_textattack-bert-base-uncased-SST-2
	done already!
Trying ngram=1_sub=-1_textattack-bert-base-uncased-SST-2__last_hidden_state_mean
	done already!
Trying ngram=1_sub=1000_bert-base-uncased
	done already!
Trying ngram=1_sub=1000_textattack-bert-base-uncased-SST-2
	done already!
Trying ngram=1_sub=100_bert-base-uncased
	done already!
Trying ngram=1_s

100%|██████████| 42/42 [00:00<00:00, 16544.02it/s]


...

Trying ngram=1_sub=-1_textattack-bert-base-uncased-imdb
	done already!
Trying ngram=2_sub=-1_textattack-bert-base-uncased-imdb
	done already!


100%|██████████| 42/42 [00:00<00:00, 17745.62it/s]


Trying ngram=1_sub=-1_ahmedrachid-FinancialBERT-Sentiment-Analysis
	done already!
Trying ngram=1_sub=-1_ahmedrachid-FinancialBERT-Sentiment-Analysis__last_hidden_state_mean
	done already!
Trying ngram=1_sub=-1_bert-base-uncased
	done already!
Trying ngram=1_sub=-1_bert-base-uncased__last_hidden_state_mean
	done already!
Trying ngram=1_sub=-1_distilbert-base-uncased__last_hidden_state_mean
	done already!
Trying ngram=1_sub=-1_yseop-distilbert-base-financial-relation-extraction__last_hidden_state_mean
	done already!
Trying ngram=2_sub=-1_ahmedrachid-FinancialBERT-Sentiment-Analysis
	done already!
Trying ngram=2_sub=-1_ahmedrachid-FinancialBERT-Sentiment-Analysis__last_hidden_state_mean
	done already!
Trying ngram=2_sub=-1_bert-base-uncased
	done already!
Trying ngram=2_sub=-1_bert-base-uncased__last_hidden_state_mean
	done already!
Trying ngram=2_sub=-1_distilbert-base-uncased__last_hidden_state_mean
	done already!
Trying ngram=2_sub=-1_yseop-distilbert-base-financial-relation-extraction

100%|██████████| 42/42 [00:00<00:00, 16726.24it/s]

Trying ngram=1_sub=-1_aatmasidha-distilbert-base-uncased-finetuned-emotion__last_hidden_state_mean
	done already!
Trying ngram=1_sub=-1_bert-base-uncased
	done already!
Trying ngram=1_sub=-1_bert-base-uncased__last_hidden_state_mean
	done already!
Trying ngram=1_sub=-1_distilbert-base-uncased__last_hidden_state_mean
	done already!
Trying ngram=1_sub=-1_nateraw-bert-base-uncased-emotion
	done already!
Trying ngram=1_sub=-1_nateraw-bert-base-uncased-emotion__last_hidden_state_mean
	done already!
Trying ngram=2_sub=-1_aatmasidha-distilbert-base-uncased-finetuned-emotion__last_hidden_state_mean
	done already!
Trying ngram=2_sub=-1_bert-base-uncased
	done already!
Trying ngram=2_sub=-1_bert-base-uncased__last_hidden_state_mean
	done already!
Trying ngram=2_sub=-1_distilbert-base-uncased__last_hidden_state_mean
	done already!
Trying ngram=2_sub=-1_nateraw-bert-base-uncased-emotion
	done already!
Trying ngram=2_sub=-1_nateraw-bert-base-uncased-emotion__last_hidden_state_mean
	done already!
Tr